In [1]:
import pandas as pd
import requests
import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)
from datetime import datetime
import seaborn as sns
import math
import matplotlib.pyplot as plt
import numpy as np
import folium
import json

## Accès API Strava

In [ ]:
auth_url = "https://www.strava.com/oauth/token"
activites_url = "https://www.strava.com/api/v3/athlete/activities"

# Obtenir la date actuelle
creation_date = datetime.now().strftime('%Y-%m-%d')

payload = {
    'client_id': "114062",
    'client_secret': "3ff69d1668764ef3f5ebdfbe71ecc04bf694d74c",
    'refresh_token': "e4a08d7904a3eedba9782ee6aa7b8644898beaff",
    'grant_type': "refresh_token",
    'f': 'json'
}

print("Requesting Token...\n")
res = requests.post(auth_url, data=payload, verify=False)
access_token = res.json()['access_token']
print("Access Token = {}\n".format(access_token))

header = {'Authorization': 'Bearer ' + access_token}
#param = {'per_page': 200, 'page': 1}
#my_dataset = requests.get(activites_url, headers=header, params=param).json()
# Liste pour stocker toutes les activités
all_activities = []

# Boucle pour récupérer toutes les pages d'activités
page = 1
while True:
    param = {'per_page': 200, 'page': page}
    activities = requests.get(activites_url, headers=header, params=param).json()

    # Si aucune activité n'est retournée, on arrête la boucle
    if not activities:
        break

    # Ajout des activités à la liste
    all_activities.extend(activities)

    # Incrémentation du numéro de page
    print(f"📄 Page {page}…")
    page += 1

# Conversion en DataFrame pandas
activities_df = pd.DataFrame(all_activities)

# Sauvegarde en fichier CSV
csv_file_path = f'../data/strava_activities_raw_{creation_date}.csv'
activities_df.to_csv(csv_file_path, index=False)

print(f"Les données ont été sauvegardées dans {csv_file_path}")


Requesting Token...

Access Token = 3fc18bbd9ca1e8ab0ed64a395cab19879798f3cd

Les données ont été sauvegardées dans ../data/strava_activities_raw_2025-07-29.csv


## Refresh quotidien Données Strava ?

## Data exploration

In [3]:
df_clean = pd.read_csv("../data/strava_activities_clean_.csv")

In [11]:
df_clean.index

RangeIndex(start=0, stop=616, step=1)

In [ ]:
activities_df.head(2)

,resource_state,athlete,name,distance,moving_time,elapsed_time,total_elevation_gain,type,sport_type,workout_type,...,elev_high,elev_low,upload_id,upload_id_str,external_id,from_accepted_tag,pr_count,total_photo_count,has_kudoed,average_cadence
0,2,"{'id': 40727595, 'resource_state': 1}",Big moustache ?,37258.4,5694,6052,283.0,Ride,Ride,10.0,...,141.0,57.4,1.624303e+10,16243025075,garmin_ping_462010212174,False,3,3,False,NaN
1,2,"{'id': 40727595, 'resource_state': 1}",Course à Dwight dans le Schrute,10017.0,2890,2893,75.0,Run,Run,0.0,...,90.4,56.2,1.622995e+10,16229954716,garmin_ping_461653472390,False,0,1,False,73.2


In [ ]:
activities_df[["map"]]

,map
0,"{'id': 'a15219932005', 'summary_polyline': '{f..."
1,"{'id': 'a15210086165', 'summary_polyline': 'ci..."
2,"{'id': 'a15197835902', 'summary_polyline': 's|..."
3,"{'id': 'a15184146738', 'summary_polyline': 's~..."
4,"{'id': 'a15163453200', 'summary_polyline': 'wv..."
...,...
608,"{'id': 'a2596458576', 'summary_polyline': '{gp..."
609,"{'id': 'a2589310060', 'summary_polyline': 'mho..."
610,"{'id': 'a2583989594', 'summary_polyline': '{mg..."
611,"{'id': 'a2332672484', 'summary_polyline': 'sip..."


In [8]:
activities_df["map"] = activities_df["map"].apply(json.dumps)


In [ ]:
activities_df[["map"]].info

In [ ]:
activities_df.columns

In [ ]:
activities_df[['sport_type','average_speed', 'max_speed', 'average_temp']]

In [ ]:
activities_df['id'].nunique()

612

In [ ]:
# Sélection des cinq premières lignes du DataFrame
first_five_activities = activities_df.head(5)

# Transposition du DataFrame pour afficher les colonnes comme des lignes
transposed_activities = first_five_activities.transpose()

# Affichage du DataFrame transposé
transposed_activities

## Data Cleaning
1. Supprimer colonnes non pertinentes
2. Renommer les colonnes pertinentes
3. Pour chaque colonne, adapter le contenu (temps, catégoriser avec les bons noms)
  A. Date en datetime
  B. temps en Km/h ET min/km
4. Ajouter une colonne qui combine Trail & CAP

In [5]:
activities_df = pd.read_csv("/Users/arthurdercq/code/Data Science/Garmin_Dashboard/data/strava_activities_raw_2025-07-29.csv")

In [6]:
# Liste des colonnes à supprimer
columns_to_drop = [
    'resource_state', 'athlete', 'type', 'workout_type', 'utc_offset',
    'location_city', 'location_state', 'location_country', 'comment_count',
    'athlete_count', 'photo_count', 'trainer', 'commute', 'manual',
    'private', 'visibility', 'flagged', 'device_watts', 'heartrate_opt_out',
    'display_hide_heartrate_option', 'upload_id', 'upload_id_str', 'external_id',
    'from_accepted_tag', 'total_photo_count'
]

# Suppression des colonnes du DataFrame
activities_df_cleaned = activities_df.drop(columns=columns_to_drop)


# Conversion de la colonne 'distance' de mètres en kilomètres
activities_df_cleaned['distance'] = activities_df_cleaned['distance'] / 1000
# Définir la colonne 'id' comme index
activities_df_cleaned = activities_df_cleaned.set_index('id')

# Conversion des colonnes 'moving_time' et 'elapsed_time' de secondes en minutes
activities_df_cleaned['moving_time'] = activities_df_cleaned['moving_time'] / 60
activities_df_cleaned['elapsed_time'] = activities_df_cleaned['elapsed_time'] / 60
# Conversion de la colonne 'average_speed' de mètres par seconde en kilomètres par heure
activities_df_cleaned['average_speed'] = activities_df_cleaned['average_speed'] * 3.6
activities_df_cleaned['max_speed'] = activities_df_cleaned['max_speed'] * 3.6
# Ajouter une nouvelle colonne 'minutes_per_km' qui convertit 'average_speed' en minutes par kilomètre
activities_df_cleaned['speed_minutes_per_km'] = 60 / activities_df_cleaned['average_speed']

def convert_minutes_to_hms(minutes):

    if minutes is None or not isinstance(minutes, (int, float)):
        return "00:00:00"
    if minutes < 0:
        minutes = abs(minutes)

    total_seconds = int(minutes * 60)
    h = total_seconds // 3600
    remainder = total_seconds % 3600
    m = remainder // 60
    s = remainder % 60
    return f"{h:02}:{m:02}:{s:02}"

# Ajouter une nouvelle colonne avec le format HH:MM:SS pour 'moving_time'
activities_df_cleaned['moving_time_hms'] = activities_df_cleaned['moving_time'].apply(convert_minutes_to_hms)

# Ajouter une nouvelle colonne avec le format HH:MM:SS pour 'elapsed_time'
activities_df_cleaned['elapsed_time_hms'] = activities_df_cleaned['elapsed_time'].apply(convert_minutes_to_hms)
total_hours = activities_df['moving_time'].sum()
total_hours
convert_minutes_to_hms(total_hours)

# Sauvegarde en fichier CSV
csv_file_path_clean = '../data/strava_activities_clean_.csv'
activities_df_cleaned.to_csv(csv_file_path_clean, index=False)

In [7]:
activities_df_cleaned.dtypes

name                     object
distance                float64
moving_time             float64
elapsed_time            float64
total_elevation_gain    float64
sport_type               object
start_date               object
start_date_local         object
timezone                 object
achievement_count         int64
kudos_count               int64
map                      object
gear_id                  object
start_latlng             object
end_latlng               object
average_speed           float64
max_speed               float64
average_temp            float64
average_watts           float64
kilojoules              float64
has_heartrate              bool
average_heartrate       float64
max_heartrate           float64
elev_high               float64
elev_low                float64
pr_count                  int64
has_kudoed                 bool
average_cadence         float64
speed_minutes_per_km    float64
moving_time_hms          object
elapsed_time_hms         object
dtype: o

In [6]:
csv_file_path_clean = '../data/strava_activities_clean_.csv'
df_clean = pd.read_csv(csv_file_path_clean)

In [7]:
print(df_clean[['moving_time', 'elapsed_time', 'achievement_count', 'kudos_count', 'pr_count']].max())


moving_time            828.016667
elapsed_time         71668.683333
achievement_count      200.000000
kudos_count             36.000000
pr_count               107.000000
dtype: float64


## Graphs

Qu'est ce que je veux montrer ? (Proposer une vision hebdo, mensuelle et annuelle)

- Volume hebomadaire
- D+ hebdo
- Répartition trail / CAP, vélo
- 


In [ ]:
def filtrer_trail_runs(df):
    # Colonnes à garder
    colonnes = ["name", "moving_time", "distance", "total_elevation_gain", "sport_type"]

    # Filtrer les colonnes si elles existent dans le DataFrame
    df = df[[col for col in colonnes if col in df.columns]]

    # Garder uniquement les lignes où sport_type est "TrailRun"
    df_filtré = df[df["sport_type"] == "TrailRun"]

    # Supprimer la colonne "sport_type" si tu ne veux plus l’avoir ensuite
    df_filtré = df_filtré.drop(columns="sport_type")

    return df_filtré


In [ ]:
def plot_elevations(df, output_file="../plots/elevations.png"):
    # Create a new figure
    plt.figure()

    # Compute activity start times (for facet ordering)
    start_times = (
        df.groupby("name").agg({"moving_time": "min"}).reset_index().sort_values("moving_time")
    )
    ncol = math.ceil(math.sqrt(len(start_times)))

    # Create facets
    p = sns.FacetGrid(
        data=df,
        col="name",
        col_wrap=ncol,
        col_order=start_times["name"],
        sharex=False,
        sharey=True,
    )

    # Add activities
    p = p.map(plt.plot, "distance", "total_elevation_gain", color="black", linewidth=4)

    # Update plot aesthetics
    p.set(xlabel=None)
    p.set(ylabel=None)
    p.set(xticks=[])
    p.set(yticks=[])
    p.set(xticklabels=[])
    p.set(yticklabels=[])
    p.set_titles(col_template="", row_template="")
    sns.despine(left=True, bottom=True)
    plt.subplots_adjust(left=0.05, bottom=0.05, right=0.95, top=0.95)
    plt.savefig(output_file)
    plt.close()

In [ ]:
de_plot_elev = filtrer_trail_runs(activities_df_cleaned)
plot_elevations(activities_df_cleaned)

In [ ]:
# select one activity
my_ride = activities_df_cleaned.iloc[0, :] # first activity (most recent)
# plot ride on map
centroid = [
    np.mean([coord[0] for coord in my_ride['map.polyline'][0]]),
    np.mean([coord[1] for coord in my_ride['map.polyline'][0]])
]
m = folium.Map(location=centroid, zoom_start=10)
folium.PolyLine(my_ride['map.polyline'], color='red').add_to(m)
display(m)